In [208]:
import numpy as np
import pandas as pd
import ms3 as ms
from preprocessing import transform_chords_abs
import warnings

# Importing development version of dimcat
import dimcat as dc

warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None

In [239]:
# Load Romantic Piano Corpus
romantic_corpus = dc.Corpus()
romantic_corpus.load("../../romantic_piano_corpus/grieg_lyric_pieces", parse_tsv=True, parse_scores=False) # make sure to parse directly from MuseScore files
#romantic_corpus.data

In [254]:
romantic_corpus.get_facet("notes")

Data object contains empty groups.


ValueError: No objects to concatenate

In [241]:
# Load Romantic Piano Corpus
abc_corpus = dc.Corpus()
abc_corpus.load("../../", parse_tsv=True, parse_scores=False) # make sure to parse directly from MuseScore files
#romantic_corpus.data

KeyboardInterrupt: 

# Helper Functions


In [218]:
# Given a corpus, process all pieces and return labels and slices
def processCorpus(corpus):
    labels = corpus.get_facet("expanded")
    # Process slices 
    salami_crp = dc.NoteSlicer().process_data(corpus)
    salami_notes = salami_crp.get_facet("notes")

    return (labels, salami_notes)

def to_pitch(midi, tpc):
    pitch_class = ms.fifths2name(tpc)
    octave = str(midi // 12)
    return pitch_class + octave 

def get_chord_offset(numeral: str, globalkey_is_minor):
    alteration = (numeral.count("#") - numeral.count("b")) * 7

    numeral = numeral.strip("#b")
    numeral = numeral.upper()#
    numeral_to_interval_major = {"I": 0, "II": 2, "III": 4, "IV": 5, "V":1, "VI":3, "VII":5}
    numeral_to_interval_minor = {"I": 0, "II": 2, "III": 9, "IV": 5, "V":1, "VI":8, "VII":10}

    if globalkey_is_minor:
        return (numeral_to_interval_minor[numeral] + alteration) % 7
    else:
        return (numeral_to_interval_major[numeral] + alteration) % 7

def interval_union(i1,i2):
    return pd.Interval(i1.left,i2.right,'left')

def transform_chords_abs(df):
    df['rootoffset'] = df.apply(lambda x: int(get_chord_offset(x.numeral,x.globalkey_is_minor)), axis = 1)

In [220]:
# Returns two dataframes, one for the chords, one for the slices
def preprocessPiece(piece : str, labels, salami_notes):
    # zoom in on the chords in one piece
    chords = labels.loc[('ABC', piece)]

    # Translate labels to absolute pitches

    desired_chord_columns = ['chord','pedal','numeral','form','figbass','changes','relativeroot','localkey','globalkey']
    chordz = chords.copy().reset_index()
    chordz = chordz.reset_index()

    chords_abs_columns = ['chord', 'globalkey','globalkey_is_minor']

    clean_chords = chordz[chordz['chord'] != '@none']
    ms.labels2global_tonic(clean_chords, inplace=True)
    clean_chords.to_csv("chordsbefore.csv")
    transform_chords_abs(clean_chords)

    # Recombine the segments with @None labels
    full_chords_abs = pd.concat([clean_chords, chordz[chordz['chord'] == '@none']]).sort_index()
    full_chords_abs.rootoffset.fillna(0, inplace=True)

    # Now we merge repeated chords
    relavant_columns = ["interval", "chord_type", "rootoffset", "globalkey"]

    dfs = pd.DataFrame()
    prev = None 
    for row in full_chords_abs[relavant_columns].iterrows():
        if prev and (row[1].chord_type == prev[1].chord_type and row[1].rootoffset == prev[1].rootoffset):
            # combine
            dfs.at[dfs.index[-1], 'interval'] = pd.Interval(dfs.iloc[-1].interval.left, row[1].interval.right, "left")
        else:
            # New row
            dfs = dfs.append(row[1])
        prev = row

    dfs.rootoffset = dfs.rootoffset.astype(int)
    full_chords_abs = dfs

    relavant_columns = [ "interval", "chord_type", "rootoffset", "globalkey"]

    full_chords_abs = full_chords_abs.reset_index()[relavant_columns]
    full_chords_abs.index.name ='segment_id'
    full_chords_abs[["chord_type", "rootoffset", "globalkey"]].to_csv('chords.csv')


    salamis = salami_notes.loc[("ABC", piece)]

    mini_salamis = salamis[['midi','tpc','tied']]
    mini_salamis['tied'] = mini_salamis['tied'].fillna(0).astype('bool')

    # Assigning each slice a segment id according to the chord.
    dfs = []
    for segment, interval in enumerate(full_chords_abs["interval"]):
        segMask = mini_salamis.index.get_level_values(0).overlaps(interval)
        slicesInInterval = mini_salamis[segMask]
        slicesInInterval.insert(0,'segment_id',segment)
        dfs.append(slicesInInterval)

    segmented_salamis = pd.concat(dfs)

    segmented_salamis['slice_id'] = pd.factorize(segmented_salamis.reset_index()['onset_slice'])[0]

    segmented_salamis['pitch'] = segmented_salamis.apply(lambda x: to_pitch(x.midi, x.tpc), axis=1)

    final_salamis_columns = ['segment_id','slice_id','pitch','tied']
    final_salamis = segmented_salamis.reset_index()[final_salamis_columns]

    final_salamis["new_segment"] = final_salamis["segment_id"].diff().astype(bool)
    final_salamis['new_slice'] = final_salamis["slice_id"].diff().astype(bool)

    # Correct the new segment and new slice fields for the first row.
    final_salamis.at[0, "new_segment"] = False
    final_salamis.at[0, "new_slice"] = False

    final_salamis.to_csv('salamis.csv',columns=["new_segment", "new_slice", "pitch","tied"], index=False)

    return (full_chords_abs[["chord_type", "rootoffset", "globalkey"]], final_salamis[["new_segment", "new_slice", "pitch", "tied"]])




In [251]:
romantic_corpus.get_facet('harmonies')

Data object contains empty groups.


ValueError: No objects to concatenate

In [238]:
abc_corpus.get_facet('expanded')

DCML_HARMONY_SYNTAX_WARNING (15,) ms3.Parse.ABC.n06op18-6_04.mscx -- /opt/homebrew/lib/python3.10/site-packages/ms3-0.5.3.post0.dev224+ga917290-py3.10.egg/ms3/annotations.py (line 350) expand_dcml():
	Score contains 1 labels that don't (and 464 that do) match the DCML standard:
	    mc  mn      label harmony_layer
	22  18  18  V7[viio64             1
DCML_HARMONY_INCOMPLETE_PEDAL_COLUMN_ERROR (13,) ms3.Parse.ABC.n06op18-6_04.mscx -- /opt/homebrew/lib/python3.10/site-packages/ms3-0.5.3.post0.dev224+ga917290-py3.10.egg/ms3/expand_dcml.py (line 155) expand_labels():
	propagate_pedal() failed with
	1 organ points started, 2 ended:
	     mc pedal   mc pedalend
	0   106     I   20        ]
	1  <NA>   NaN  108        ]
DCML_HARMONY_INCOMPLETE_PEDAL_COLUMN_ERROR (13,) ms3.Parse.ABC.n10op74_04.mscx -- /opt/homebrew/lib/python3.10/site-packages/ms3-0.5.3.post0.dev224+ga917290-py3.10.egg/ms3/expand_dcml.py (line 155) expand_labels():
	propagate_pedal() failed with
	5 organ points started, 6 ended

quarterbeats  duration_qb   mc   mn  \
corpus fname        interval                                               
ABC    n01op18-1_01 [0.0, 3.0)                  0          3.0    1    1   
                    [3.0, 6.0)                  3          3.0    2    2   
                    [6.0, 9.0)                  6          3.0    3    3   
                    [9.0, 15.0)                 9          6.0    4    4   
                    [15.0, 18.0)               15          3.0    6    6   
...                                           ...          ...  ...  ...   
       n16op135_04  [1140.0, 1141.0)         1140          1.0  281  280   
                    [1141.0, 1142.0)         1141          1.0  281  280   
                    [1142.0, 1144.0)         1142          2.0  282  281   
                    [1144.0, 1146.0)         1144          2.0  282  281   
                    [1146.0, 1150.0)         1146          4.0  283  282   

                                     mc_onset mn_onset timesig  staff  voice  \
corpus fname        interval                                                   
ABC    n01op18-1_01 [0.0, 3.0)              0        0     3/4      4      1   
                    [3.0, 6.0)              0        0     3/4      4      1   
                    [6.0, 9.0)              0        0     3/4      4      1   
                    [9.0, 15.0)             0        0     3/4      4      1   
                    [15.0, 18.0)            0        0     3/4      4      1   
...                                       ...      ...     ...    ...    ...   
       n16op135_04  [1140.0, 1141.0)      1/2      1/2     4/4      4      1   
                    [1141.0, 1142.0)      3/4      3/4     4/4      4      1   
                    [1142.0, 1144.0)        0        0     4/4      4      1   
                    [1144.0, 1146.0)      1/2      1/2     4/4      4      1   
                    [1146.0, 1150.0)        0        0     4/4      4      1   

                                      volta  ... phraseend chord_type  \
corpus fname        interval                 ...                        
ABC    n01op18-1_01 [0.0, 3.0)         <NA>  ...       NaN          M   
                    [3.0, 6.0)         <NA>  ...       NaN          M   
                    [6.0, 9.0)         <NA>  ...       NaN          M   
                    [9.0, 15.0)        <NA>  ...       NaN          M   
                    [15.0, 18.0)       <NA>  ...       NaN        Mm7   
...                                     ...  ...       ...        ...   
       n16op135_04  [1140.0, 1141.0)   <NA>  ...       NaN          M   
                    [1141.0, 1142.0)   <NA>  ...       NaN          M   
                    [1142.0, 1144.0)   <NA>  ...       NaN          M   
                    [1144.0, 1146.0)   <NA>  ...       NaN        Mm7   
                    [1146.0, 1150.0)   <NA>  ...        \\          M   

                                     globalkey_is_minor localkey_is_minor  \
corpus fname        interval                                                
ABC    n01op18-1_01 [0.0, 3.0)                    False             False   
                    [3.0, 6.0)                    False             False   
                    [6.0, 9.0)                    False             False   
                    [9.0, 15.0)                   False             False   
                    [15.0, 18.0)                  False             False   
...                                                 ...               ...   
       n16op135_04  [1140.0, 1141.0)               True             False   
                    [1141.0, 1142.0)               True             False   
                    [1142.0, 1144.0)               True             False   
                    [1144.0, 1146.0)               True             False   
                    [1146.0, 1150.0)               True             False   

                                        chord_tones add

In [242]:
(rchord, rlabels) = processCorpus(romantic_corpus)

Data object contains empty groups.


ValueError: No objects to concatenate

In [221]:
pieces = labels.loc["ABC"].index.unique(0).tolist()

NameError: name 'labels' is not defined

In [36]:
for piece in pieces:
    (chords, slices) = preprocessPiece(piece, labels)
    chords.to_csv("inputs/chords/{}.csv".format(piece))
    slices.to_csv("inputs/slices/{}.csv".format(piece),index=False)

# EXPERIMENTS

In [8]:
import subprocess
import json

In [202]:
def runFullParse (chordsPath:str, slicesPath:str, jsonPath:str):
    cmd = ["stack","run","fullParse","--",chordsPath, slicesPath, "preprocessing/" + jsonPath] 
    print("Running command: " + (" ".join(cmd)))
    print("Expecting results in " + jsonPath)
    #res = subprocess.run(cmd, cwd="..")
    res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")

    if res.returncode != 0:
        print("Error in subprocess")
        print(res.stderr)
        print(res.stdout)
        return;
    else:
        f = open(jsonPath)
        results = json.load(f)
        f.close()
        return results
     

In [205]:
def runExperiment(piece:str): 
    return runFullParse("preprocessing/inputs/chords/{}.csv".format(piece),"preprocessing/inputs/slices/{}.csv".format(piece), "temp/{}.json".format(piece))

In [207]:
runExperiment("shortest") 

Running command: stack run fullParse -- preprocessing/inputs/chords/shortest.csv preprocessing/inputs/slices/shortest.csv preprocessing/temp/shortest.json
Expecting results in temp/shortest.json


{'Seg by Seg': -14.419334585957994, 'nemjef': 88}

In [ ]:
def runFullPieceExperiment(piece:str):
    # Runs Heuristic search, Random Search, and Random sample search on the given piece.
    # Returns the scores for each 


In [1]:
def runExperiments():
    # Run experiments 

    # EXPERIMENT 1:
    # Heuristic Search
    #  


SyntaxError: incomplete input (3626526487.py, line 5)